In [2]:
from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, \
    squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import inv
from scipy.stats import chi2
from pandas import read_csv
import statsmodels.api as sm
import os

In [3]:
os.getcwd()


'/Users/mandarphatak'

In [4]:
os.chdir("/Users/mandarphatak/Downloads/")

In [5]:
data = read_csv('fama_french.csv')

data

,date,rf,mkt_excess,smb,hml,cci,cci_mkt,IYW,IYR,IYH,IYF,IYE,IYK,IYJ,IYC,IDU,IYM,IYZ
0,2000-07-01,0.0048,-0.0251,-0.0307,0.0831,-5.5,0.13805,-0.064414,0.057434,-0.053015,0.078780,-0.042459,0.042152,-0.064926,0.003154,0.069376,0.012847,-0.089072
1,2000-08-01,0.0050,0.0703,-0.0061,-0.0139,3.8,0.26714,0.137438,-0.015082,0.025500,0.096563,0.117141,-0.044754,0.124863,0.001048,0.134341,0.015041,-0.024803
2,2000-09-01,0.0051,-0.0545,-0.0182,0.0717,-2.2,0.11990,-0.151668,0.031050,0.040956,0.032090,0.016291,0.004608,-0.041565,-0.025131,0.091373,-0.103727,-0.053180
3,2000-10-01,0.0056,-0.0276,-0.0388,0.0571,1.7,-0.04692,-0.080775,-0.056511,0.021302,-0.012531,-0.042117,0.040054,-0.053262,-0.003988,-0.041515,0.112744,0.012110
4,2000-11-01,0.0051,-0.1072,-0.0341,0.1230,-6.7,0.71824,-0.234329,0.020553,0.023864,-0.049229,-0.045122,-0.001474,-0.059331,-0.069580,-0.000754,-0.025363,-0.186215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2023-07-01,0.0045,0.0321,0.0208,0.0411,7.3,0.23433,0.044089,0.017449,0.007172,0.064335,0.079132,0.019921,0.029423,0.026275,0.018716,0.042315,0.014480
277,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,-10.9,0.26051,-0.019354,-0.030778,-0.003755,-0.032112,0.013411,-0.038232,-0.014841,-0.028478,-0.060038,-0.035333,0.019625
278,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-3.1,0.16244,-0.057964,-0.073956,-0.035478,-0.022274,0.019474,-0.041836,-0.059119,-0.054742,-0.052082,-0.050347,-0.063592
279,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-0.4,0.01276,-0.011437,-0.035961,-0.036879,-0.029294,-0.055450,-0.015924,-0.036277,-0.034653,0.013183,-0.032457,-0.028169


In [6]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','cci','cci_mkt']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [8]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params


In [9]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [10]:
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)

In [11]:
Jpval

0.07163989260422166

In [17]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB         HML     CCI  CCI_MKT')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}    {2:0.4f}   {3:0.4f}   {4:0.4f}'.format(arp[0], arp[1], arp[2], arp[3], arp[4]))
print('Std. Err.  {0:0.4f}    {1:0.4f}    {2:0.4f}   {3:0.4f}   {4:0.4f}'.format(arpSE[0], arpSE[1], arpSE[2], arpSE[3], arpSE[4]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2], b[3],b[4]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}'.format(s[0], s[1], s[2], s[3],s[4],s[5]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f}   {:>10,.4f} {:>10,.4f}'.format(t[0], t[1], t[2], t[3],t[4],t[5]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB         HML     CCI  CCI_MKT
---------------------------------------------------------------
Premia     0.0649    0.0029    0.0099   4.1776   -2.0877
Std. Err.  0.0097    0.0134    0.0096   6.1628   0.4707



J-test:   18.4534
P-value:   0.0716
Size: 1, Value:1   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)
Coefficients:    -0.0000      1.3454      0.0696     -0.6398    -0.0004      0.0077
Std Err.          0.0018      0.0491      0.0928      0.0757      0.0003     0.0043
T-stat           -0.0264     27.3821      0.7506     -8.4569      -1.2620     1.8163

Size: 2, Value:1   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)
Coefficients:     0.0016      0.8199      0.1694      0.3869     0.0009     -0.0055
Std Err.          0.0024      0.0737      0.1075      0.0899      0.0004     0.0104
T-stat            0.6699     11.1306      1.5756      4.3040       2.0520    -0.5270

Size: 3, Value:1  

In [33]:
### if we use the linear GMM library "IYW", "IYR" , "IYH" , "IYF" , "IYE" , "IYK",  "IYJ" , "IYC" , "IDU" , "IYM" , "IYZ"

from linearmodels.asset_pricing import LinearFactorModelGMM
data = read_csv('fama_french.csv')
data = data.dropna()
data

,date,rf,mkt_excess,smb,hml,cci,cci_mkt,IYW,IYR,IYH,IYF,IYE,IYK,IYJ,IYC,IDU,IYM,IYZ
0,2000-07-01,0.0048,-0.0251,-0.0307,0.0831,-5.5,0.13805,-0.064414,0.057434,-0.053015,0.078780,-0.042459,0.042152,-0.064926,0.003154,0.069376,0.012847,-0.089072
1,2000-08-01,0.0050,0.0703,-0.0061,-0.0139,3.8,0.26714,0.137438,-0.015082,0.025500,0.096563,0.117141,-0.044754,0.124863,0.001048,0.134341,0.015041,-0.024803
2,2000-09-01,0.0051,-0.0545,-0.0182,0.0717,-2.2,0.11990,-0.151668,0.031050,0.040956,0.032090,0.016291,0.004608,-0.041565,-0.025131,0.091373,-0.103727,-0.053180
3,2000-10-01,0.0056,-0.0276,-0.0388,0.0571,1.7,-0.04692,-0.080775,-0.056511,0.021302,-0.012531,-0.042117,0.040054,-0.053262,-0.003988,-0.041515,0.112744,0.012110
4,2000-11-01,0.0051,-0.1072,-0.0341,0.1230,-6.7,0.71824,-0.234329,0.020553,0.023864,-0.049229,-0.045122,-0.001474,-0.059331,-0.069580,-0.000754,-0.025363,-0.186215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2023-07-01,0.0045,0.0321,0.0208,0.0411,7.3,0.23433,0.044089,0.017449,0.007172,0.064335,0.079132,0.019921,0.029423,0.026275,0.018716,0.042315,0.014480
277,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,-10.9,0.26051,-0.019354,-0.030778,-0.003755,-0.032112,0.013411,-0.038232,-0.014841,-0.028478,-0.060038,-0.035333,0.019625
278,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-3.1,0.16244,-0.057964,-0.073956,-0.035478,-0.022274,0.019474,-0.041836,-0.059119,-0.054742,-0.052082,-0.050347,-0.063592
279,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-0.4,0.01276,-0.011437,-0.035961,-0.036879,-0.029294,-0.055450,-0.015924,-0.036277,-0.034653,0.013183,-0.032457,-0.028169


In [31]:
data.iloc[:, 7:] = data.iloc[:, 7:].values - data[["rf"]].values

In [32]:
portfolios= data[["IYW", "IYR" , "IYH" , "IYF" , "IYE" , "IYK",  "IYJ" , "IYC" , "IDU" , "IYM" , "IYZ"]]
portfolios


,IYW,IYR,IYH,IYF,IYE,IYK,IYJ,IYC,IDU,IYM,IYZ
0,-0.069214,0.052634,-0.057815,0.073980,-0.047259,0.037352,-0.069726,-0.001646,0.064576,0.008047,-0.093872
1,0.132438,-0.020082,0.020500,0.091563,0.112141,-0.049754,0.119863,-0.003952,0.129341,0.010041,-0.029803
2,-0.156768,0.025950,0.035856,0.026990,0.011191,-0.000492,-0.046665,-0.030231,0.086273,-0.108827,-0.058280
3,-0.086375,-0.062111,0.015702,-0.018131,-0.047717,0.034454,-0.058862,-0.009588,-0.047115,0.107144,0.006510
4,-0.239429,0.015453,0.018764,-0.054329,-0.050222,-0.006574,-0.064431,-0.074680,-0.005854,-0.030463,-0.191315
...,...,...,...,...,...,...,...,...,...,...,...
276,0.039589,0.012949,0.002672,0.059835,0.074632,0.015421,0.024923,0.021775,0.014216,0.037815,0.009980
277,-0.023854,-0.035278,-0.008255,-0.036612,0.008911,-0.042732,-0.019341,-0.032978,-0.064538,-0.039833,0.015125
278,-0.062264,-0.078256,-0.039778,-0.026574,0.015174,-0.046136,-0.063419,-0.059042,-0.056382,-0.054647,-0.067892
279,-0.016137,-0.040661,-0.041579,-0.033994,-0.060150,-0.020624,-0.040977,-0.039353,0.008483,-0.037157,-0.032869


In [34]:
factors = data[["mkt_excess", "smb", "hml", "cci","cci_mkt"]]

In [35]:
mod = LinearFactorModelGMM(portfolios, factors)
res = mod.fit()
print(res)

Iteration: 0, Objective: 29.826139652294454
Iteration: 10, Objective: 23.857508157528354
Iteration: 20, Objective: 21.62240654196041
Iteration: 30, Objective: 17.686521605390098
Iteration: 40, Objective: 14.532044213062182
Iteration: 50, Objective: 13.582574997044437
Iteration: 60, Objective: 11.867734543537695
Iteration: 70, Objective: 10.439213746138643
Iteration: 80, Objective: 10.03286543393414
Iteration: 90, Objective: 10.029283002523337
         Current function value: 10.029282
         Iterations: 97
         Function evaluations: 8856
         Gradient evaluations: 134
Iteration: 0, Objective: 9.20046322366795
Iteration: 10, Objective: 8.930430847938625
Iteration: 20, Objective: 8.780953982320622
Iteration: 30, Objective: 8.403251474871341
Iteration: 40, Objective: 8.227945325656428
Iteration: 50, Objective: 8.125098443909865
Iteration: 60, Objective: 8.05998479048805
Iteration: 70, Objective: 8.008438232902737
         Current function value: 8.007505
         Iterations: 75


In [36]:
res = mod.fit(cov_type="kernel", kernel="bartlett", disp=0)
print(res)

                    LinearFactorModelGMM Estimation Summary                     
No. Test Portfolios:                 11   R-squared:                      0.6782
No. Factors:                          5   J-statistic:                    6.1621
No. Observations:                   281   P-value                         0.4053
Date:                  Thu, Jan 04 2024   Distribution:                  chi2(6)
Time:                          09:01:19                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0185     0.0427     0.4326     0.6653     -0.0653      0.1023
smb            0.0087     0.0526    

In [37]:
res.full_summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                    LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 11   R-squared:                      0.6782
No. Factors:                          5   J-statistic:                    6.1621
No. Observations:                   281   P-value                         0.4053
Date:                  Thu, Jan 04 2024   Distribution:                  chi2(6)
Time:                          09:01:19                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0185     0.0427     0.4326     0.6653     -0.0653      0.1023
smb            0.0087     0.0526     0.1646     0.8692     -0.0945      0.1118
hml           -0.0085     0.0366    -0.2328     0.8159     -0.0803      0.0633
cci            47.233     194.44     0.2429     0.8081     -333.87      428.34
cci_mkt       -5.3976     22.260    -0.2425     0.8084     -49.026      38.231


                               IYW Coefficients                               
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
alpha          0.0001     0.0017     0.0818     0.9348     -0.0033      0.0036
mkt_excess     1.3441     0.0024     553.17     0.0000      1.3393      1.3489
smb            0.0366     0.0015     23.767     0.0000      0.0336      0.0396
hml           -0.6616     0.0012    -550.98     0.0000     -0.6640     -0.6593
cci           -0.0002     0.0030    -0.0779     0.9379     -0.0062      0.0057
cci_mkt        0.0020     0.0014     1.4724     0.1409     -0.0007      0.0047


                               IYR Coefficients                               
==============================================================================
alpha         -0.0005     0.0010    -0.4753     0.6346     -0.0026      0.0016
mkt_excess     0.7623     0.0011     672.82     0.0000      0.7601      0.7645
smb            0.1540     0.0022     70.478     0.0000      0.1497      0.1583
hml            0.2954     0.0021     141.53     0.0000      0.2913      0.2994
cci            0.0005     0.0023     0.2373     0.8124     -0.0040      0.0051
cci_mkt        0.0054     0.0557     0.0971     0.9226     -0.1038      0.1146


                               IYH Coefficients                               
==============================================================================
alpha       4.519e-05     0.0937     0.0005     0.9996     -0.1835      0.1836
mkt_excess     0.6670     0.0859     7.7678     0.0000      0.4987      0.8353
smb           -0.2568     0.0002    -1033.4     0.0000     -0.2573     -0.2563
hml            0.0106     0.0031     3.4397     0.0006      0.0045      0.0166
cci           -0.0001     0.0724    -0.0017     0.9986     -0.1421      0.1418
cci_mkt       -0.0005     0.0977    -0.0055     0.9956     -0.1919      0.1909


                               IYF Coefficients                               
==============================================================================
alpha         -0.0004     0.1063    -0.0036     0.9971     -0.2087      0.2080
mkt_excess     1.0684     0.0003     3711.6     0.0000      1.0678      1.0690
smb           -0.0532     0.0038    -13.955     0.0000     -0.0607     -0.0458
hml            0.5716     0.0343     16.662     0